In [165]:
import json
from requests_oauthlib import OAuth1Session
import pandas
import requests
pandas.set_option("display.max_colwidth", 200)
import numpy

In [3]:
CONSUMER_KEY = ''
CONSUMER_SECRET = ''
ACCESS_TOKEN = ''
ACCESS_TOKEN_SECRET = ''
twitter = OAuth1Session(CONSUMER_KEY,CONSUMER_SECRET,ACCESS_TOKEN,ACCESS_TOKEN_SECRET)

In [110]:
GOOGLE_API_KEY = ''


In [126]:
def analyze_sentiment(analyze_text):
    url = f'https://language.googleapis.com/v1beta2/documents:analyzeSentiment?key={GOOGLE_API_KEY}'
    header = {'Content-Type':'application/json'}
    body = {
        "document":{
            "type" : "PLAIN_TEXT",
            "language" : "JA",
            "content" : analyze_text
        }
    }
    res = requests.post(url, headers=header,json=body)
    if (res.status_code == 200):
            result_analyze_sentiment = json.loads(res.text)
            return result_analyze_sentiment

In [111]:
# 特定ユーザーのツイートを取得
def get_user_tweet_dataframe(search_twitter_id):
    url = 'https://api.twitter.com/1.1/statuses/user_timeline.json'
    #最大2200件のツイートを取得するためのページ
    pages = [1,2,3,4,5,6,7,8,9,10,11]
    df_timelines_concat = pandas.DataFrame()
    for page in pages:

        params = {'screen_name':search_twitter_id,
                'count':200,
                'page':page,
                'include_entities' : True,
                'exclude_replies' : False,
                'include_rts' : False
                }
        res = twitter.get(url,params=params)
        if (res.status_code == 200):
            timelines = json.loads(res.text)
            df_timelines = pandas.DataFrame(timelines)
            df_timelines_concat = pandas.concat([df_timelines_concat,df_timelines],axis=0)
        
    return df_timelines_concat

In [112]:
# 特定のユーザーへのリプライのみ取得
def get_user_replies_tweet_dataframe(search_twitter_id,search_replies_twitter_id_list=[]):
    df_tweet = get_user_tweet_dataframe(search_twitter_id)
    df_reply_tweet = df_tweet.dropna(subset=['in_reply_to_screen_name'])
    if (len(search_replies_twitter_id_list)>0):
        df_reply_user_tweet = df_reply_tweet[df_reply_tweet['in_reply_to_screen_name'].isin(search_replies_twitter_id_list)].copy()
        for search_replies_twitter_id in search_replies_twitter_id_list:
            df_reply_user_tweet.text = df_reply_user_tweet.text.replace(f'@{search_replies_twitter_id}', '', regex=True)
        return df_reply_user_tweet
    
    else:
        return df_reply_tweet

In [129]:
search_twitter_id = 'twitter_account_id'
search_replies_twitter_id_list = ['twitter_account_id']
df_tweet = get_user_replies_tweet_dataframe(search_twitter_id,search_replies_twitter_id_list)[['created_at','text']].copy()

In [132]:
# 取得したツイートを感情分析
analyze_sentiment_list = []
for tweet_text in df_tweet['text'].values:
    analyze_sentiment_list.append(analyze_sentiment(tweet_text))

In [180]:
#感情分析結果をdataframeにセット
df_analyze_sentiment = pandas.DataFrame(analyze_sentiment_list)

In [185]:
df_analyze_sentiment['magnitude'] = 0.00
df_analyze_sentiment['score'] = 0.00

# 文章感情分析のスコアを列にセット
for index, row in df_analyze_sentiment.iterrows():
    df_analyze_sentiment.at[index, 'magnitude'] += row['documentSentiment']['magnitude']
    df_analyze_sentiment.at[index, 'score'] += row['documentSentiment']['score']

df_analyze_sentiment['magnitude_score'] = (df_analyze_sentiment['magnitude'] * df_analyze_sentiment['score']).abs()

In [186]:
# 感情分析をセンテンス毎に分解
shaping_sentences_sentiment_dic_list = []
for sentences_sentiment_list in df_analyze_sentiment['sentences'].values:
    for sentences_sentiment_dic in sentences_sentiment_list:
        shaping_sentences_sentiment_dic = {'content':sentences_sentiment_dic['text']['content'],
                                                                'magnitude':sentences_sentiment_dic['sentiment']['magnitude'],
                                                                'score':sentences_sentiment_dic['sentiment']['score']
                                                            }
        shaping_sentences_sentiment_dic_list.append(shaping_sentences_sentiment_dic)
df_analyze_sentences_sentiment = pandas.DataFrame(shaping_sentences_sentiment_dic_list)